In [2]:
# Import dependencies
import pandas as pd

In [3]:
# Load csv files
school_data_csv="Resources/schools_complete.csv"
student_data_csv="Resources/students_complete.csv"

# Read csv and store into Pandas DataFrame
school_data=pd.read_csv(school_data_csv)
student_data=pd.read_csv(student_data_csv)

# Combine into a signle dataset
dataset=pd.merge(school_data, student_data, how="left", on=["school_name", "school_name"])
dataset.head()


,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


# District Summary

In [4]:
# CREATE a high-level snapshot of the district's key metrics in a DataFrame, including the following:
# Total schools
total_schools=len(dataset["school_name"].unique())
# Total students
total_students=dataset["student_name"].count()
# Total budget
total_budget=school_data["budget"].sum()

# Average math score
avg_math=dataset["math_score"].mean()
# Average reading score
avg_read=dataset["reading_score"].mean()

# % passing math
pass_math=dataset[dataset["math_score"]>=70].count()["student_name"]
percentage_math=(pass_math/total_students)*100
# % passing reading
pass_read=dataset[dataset["reading_score"]>=70].count()["student_name"]
percentage_read=(pass_read/total_students)*100


# % overall passing
overall_pass=dataset[(dataset["math_score"] >=70) & (dataset["reading_score"] >=70)]
overall_count=overall_pass["student_name"].count()
overall_percentage=(overall_count/total_students)*100

# Create District Summary DataFrame
district_summary_df=pd.DataFrame(
    [{"Total Schools": total_schools, 
      "Total Students": total_students,
      "Total Budget": total_budget,
      "Average Math Score": avg_math,
      "Average Reading Score": avg_read,
      "% Passing Math": percentage_math,
      "% Passing Reading": percentage_read,
      "% Overall Passing": overall_percentage}])

district_summary_df=district_summary_df[["Total Schools", "Total Students", "Total Budget", 
                                         "Average Math Score", "Average Reading Score", 
                                         "% Passing Math", "% Passing Reading", 
                                         "% Overall Passing"]]

district_summary_df["Total Students"]=district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df.head()



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [36]:
# CREATE a DataFrame that summarizes key metrics about each school, including the following:
# School type
school_types=school_data.set_index(["school_name"])["type"]
# Total students
students_per_school=dataset["school_name"].value_counts()

# Total school budget
budget_per_school=dataset.groupby(["school_name"]).mean()["budget"]
# Per student budget
per_student_budget=budget_per_school/students_per_school

# Average math score (ams)
ams=dataset.groupby(["school_name"]).mean()["math_score"]
# Average reading score (ars)
ars=dataset.groupby(["school_name"]).mean()["reading_score"]

# % passing math (ppm), passing math (pm)
pm=dataset[(dataset["math_score"]>=70)]
pm_per_school=pm.groupby(["school_name"]).count()["student_name"]
ppm=(pm_per_school/students_per_school)*100

# % passing reading (ppr), passing reading (pr)
pr=dataset[(dataset["reading_score"]>=70)]
pr_per_school=pr.groupby(["school_name"]).count()["student_name"]
ppr=(pr_per_school/students_per_school)*100

# % overall passing (pop), passing math & reading (pmr)
pmr=dataset[(dataset["math_score"]>=70) & (dataset["reading_score"]>=70)]
pmr_per_school=pmr.groupby(["school_name"]).count()["student_name"]
pop=(pmr_per_school/students_per_school)*100

# CREATE School Summary DataFrame
school_summary_df=pd.DataFrame({"School Type":school_types,
        "Total Students":students_per_school,
        "Total School Budget":budget_per_school,
        "Per Student Budget":per_student_budget,
        "Average Math Score":ams,
        "Average Reading Score":ars,
        "% Passing Math":ppm,
        "% Passing Reading":ppr,
        "% Overall Passing":pop})

school_summary_df=school_summary_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget", 
                                     "Average Math Score", "Average Reading Score",
                                     "% Passing Math", "% Passing Reading", 
                                     "% Overall Passing"]]

school_summary_df["Total School Budget"]=school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"]=school_summary_df["Per Student Budget"].map("${:,.2f}".format)
school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


# Highest-Performing Schools (by % Overall Passing) 

In [13]:
# CREATE a DataFrame that highlights the top-5 performing schools based on "% Overall Passing", include the following:
# Highest-Performing Schools (hps)
hps_df=school_summary_df.sort_values(["% Overall Passing"], ascending=False)
hps_df.head()
# CREATE Highest-Performing Schools DataFrame

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Lowest-Performing Schools (by % Overall Passing)

In [15]:
# CREATE a DataFrame that highlights the bottom-5 performing schools based on "$ Overall Passing", include the following:
# Lowest-Performing Schools (lps)
lps_df=school_summary_df.sort_values(["% Overall Passing"], ascending=True)
lps_df.head()
# CREATE Lowest-Performing Schools DataFrame

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Math Scores by Grade

In [29]:
# CREATE a DataFrame that lists the average math score for students in grade level (9th, 10th, 11th, and 12th) each school.
# 9th 
ninth=dataset[(dataset["grade"]=="9th")]
ninth_scores=ninth.groupby(["school_name"]).mean()["math_score"]

# 10th 
tenth=dataset[(dataset["grade"]=="10th")]
tenth_scores=tenth.groupby(["school_name"]).mean()["math_score"]

# 11th 
eleventh=dataset[(dataset["grade"]=="11th")]
eleventh_scores=eleventh.groupby(["school_name"]).mean()["math_score"]

# 12th 
twelfth=dataset[(dataset["grade"]=="12th")]
twelfth_scores=twelfth.groupby(["school_name"]).mean()["math_score"]

# CREATE Math Scores by Grade DataFrame
# Math Scores by Grade (msg)
msg_df=pd.DataFrame({"9th": ninth_scores,
                     "10th": tenth_scores,
                     "11th": eleventh_scores,
                     "12th": twelfth_scores})

msg_df=msg_df[["9th", "10th", "11th", "12th"]]
msg_df.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


# Reading Scores by Grade

In [30]:
# CREATE a DataFrame that lists the average reading score for students in grade level (9th, 10th, 11th, and 12th) each school.
# 9th 
nine_g=dataset[(dataset["grade"]=="9th")]
nine_g_scores=nine_g.groupby(["school_name"]).mean()["reading_score"]

# 10th  "tenth graders" (tg)
ten_g=dataset[(dataset["grade"]=="10th")]
ten_g_scores=ten_g.groupby(["school_name"]).mean()["reading_score"]

# 11th 
ele_g=dataset[(dataset["grade"]=="11th")]
ele_g_scores=ele_g.groupby(["school_name"]).mean()["reading_score"]

# 12th 
twe_g=dataset[(dataset["grade"]=="12th")]
twe_g_scores=twe_g.groupby(["school_name"]).mean()["reading_score"]

# CREATE Reading Scores by Grade DataFrame
# Reading Scores by Grade (rsg)
rsg_df=pd.DataFrame({"9th": nine_g_scores,
                     "10th": ten_g_scores,
                     "11th": ele_g_scores,
                     "12th": twe_g_scores})

rsg_df=rsg_df[["9th", "10th", "11th", "12th"]]
rsg_df.head()


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


# Scores by School Spending

In [64]:
# CREATE a table that breaks down school performance based on average spending ranges (per students). Create bins(4) w/reasonable cutoff values
    # to group school spending, include the following:
school_summary_df.sort_values(["Per Student Budget"], ascending=True).head(15)
bins=[0, 585, 630, 645, 680]
group_names=["<$585", "$585-630", "$630-645", "$645-680"]
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_student_budget, bins, labels=group_names, include_lowest=True)

# Average math score (ams)
spending_ams=round(school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"],2)
# Average reading score (ars)
spending_ars=round(school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"],2)
# % passing math (ppm)
spending_ppm=round(school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"],2)
# % passing reading (ppr)
spending_ppr=round(school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"],2)
# % overall passing (pop)
spending_pop=round(school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"],2)

# CREATE Scores by School Spending Table (sst)
spending_table=pd.DataFrame({"Average Math Score":spending_ams,
    "Average Reading Score":spending_ars,
    "% Passing math":spending_ppm,
    "% Passing Reading":spending_ppr,
    "% Overall Passing":spending_pop })

spending_table=spending_table[["Average Math Score",
         "Average Reading Score",
         "% Passing math", 
         "% Passing Reading",
         "% Overall Passing"]]
spending_table.head()


,Average Math Score,Average Reading Score,% Passing math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


# Scores by School Size

In [63]:
# CREATE a table that breaks down school performance based on school size (small, medium, large)
size_bins=[0, 1000, 2000, 5000]
size_group_names=["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_summary_df["School Size"] = pd.cut(students_per_school, size_bins, labels=size_group_names, include_lowest=True)

# Average math score (ams)
size_ams=school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
# Average reading score (ars)
size_ars=school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
# % passing math (ppm)
size_ppm=school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
# % passing reading (ppr)
size_ppr=school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
# % overall passing (pop)
size_pop=school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

# CREATE Scores by School Size Table
size_table=pd.DataFrame({"Average Math Score":size_ams,
    "Average Reading Score":size_ars,
    "% Passing math":size_ppm,
    "% Passing Reading":size_ppr,
    "% Overall Passing":size_pop })

size_table=size_table[["Average Math Score",
         "Average Reading Score",
         "% Passing math", 
         "% Passing Reading",
         "% Overall Passing"]]
size_table.head()

,Average Math Score,Average Reading Score,% Passing math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


# Scores by School Type

In [65]:
# CREATE a table that breaks down school performance based on school type (district or charter).
# Average math score (ams)
type_ams=school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
# Average reading score (ars)
type_ars=school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
# % passing math (ppm)
type_ppm=school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
# % passing reading (ppr)
type_ppr=school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
# % overall passing (pop)
type_pop=school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

# CREATE Scores by School Type Table
type_table=pd.DataFrame({"Average Math Score":type_ams,
    "Average Reading Score":type_ars,
    "% Passing math":type_ppm,
    "% Passing Reading":type_ppr,
    "% Overall Passing":type_pop })

type_table=type_table[["Average Math Score",
         "Average Reading Score",
         "% Passing math", 
         "% Passing Reading",
         "% Overall Passing"]]
type_table.head()

,Average Math Score,Average Reading Score,% Passing math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
